<a href="https://colab.research.google.com/github/sahinMT/opencv/blob/master/LSTM_Trend_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === GEREKLİ KÜTÜPHANELER ===
!pip install git+https://github.com/rongardF/tvdatafeed openpyxl tqdm ta tensorflow scikit-learn
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from ta.trend import EMAIndicator, SMAIndicator, ADXIndicator, MACD, CCIIndicator
from ta.momentum import RSIIndicator
from tqdm import tqdm
from sklearn.metrics import classification_report
import json

# === GLOBAL PARAMETRELER ===
features = ['close', 'high', 'low', 'volatility', 'rwi_up', 'rwi_down', 'darvas_top', 'darvas_bottom']

def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('1 Saat',    Interval.in_1_hour),
        '4': ('2 Saat',    Interval.in_2_hour),
        '5': ('4 Saat',    Interval.in_4_hour),
        '6': ('Günlük',    Interval.in_daily),
        '7': ('Haftalık',  Interval.in_weekly),
        '8': ('Aylık',     Interval.in_monthly),
    }
    print("Zaman dilimini seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}: {name}")
    choice = input("Seçiminiz (varsayılan: 6): ").strip()
    return intervals.get(choice, intervals['6'])[1]

def get_bar_ahead(interval):
    if interval in [Interval.in_15_minute, Interval.in_30_minute]:
        return 8
    elif interval in [Interval.in_1_hour, Interval.in_2_hour]:
        return 6
    elif interval == Interval.in_4_hour:
        return 5
    elif interval == Interval.in_daily:
        return 5
    elif interval == Interval.in_weekly:
        return 4
    elif interval == Interval.in_monthly:
        return 2
    return 5

def get_bist_symbols():
    try:
        symbols = get_all_symbols(market='turkey')
        return [s.replace('BIST:', '') for s in symbols]
    except Exception as e:
        print("Sembol hatası:", e)
        return []

def fetch_and_prepare_data(symbols, interval, bar_ahead):
    tv = TvDatafeed()
    all_data = []
    for symbol in tqdm(symbols, desc="Veri Alınıyor", ncols=80):
        try:
            df = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=400)
            if df is None or df.empty or len(df) < 50:
                continue
            df['symbol'] = symbol
            df['darvas_top'] = df['high'].rolling(20).max().shift(1)
            df['darvas_bottom'] = df['low'].rolling(20).min().shift(1)
            df['volatility'] = df['close'].rolling(10).std()
            atr = df['volatility']
            rwi_length = 14
            rwi_sqrt = np.sqrt(rwi_length)
            df['rwi_up'] = (df['high'].rolling(rwi_length).max() - df['low']) / (atr * rwi_sqrt)
            df['rwi_down'] = (df['high'] - df['low'].rolling(rwi_length).min()) / (atr * rwi_sqrt)
            df['label'] = (df['close'].shift(-bar_ahead) > df['close']).astype(int)
            df.dropna(inplace=True)
            all_data.append(df)
        except Exception as e:
            print(f"{symbol} verisi alınamadı: {e}")
    return pd.concat(all_data) if all_data else pd.DataFrame()

def prepare_lstm_data(df, feature_cols, bar_ahead=5, lookback=20):
    df = df.copy()
    df.dropna(inplace=True)
    scaler = MinMaxScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols])

    X, y, symbols = [], [], []
    for i in range(lookback, len(df) - bar_ahead):
        X.append(df[feature_cols].iloc[i - lookback:i].values)
        y.append(df['label'].iloc[i])
        symbols.append(df['symbol'].iloc[i])

    return np.array(X), np.array(y), symbols, scaler

def build_and_train_lstm(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = Sequential([
        LSTM(64, input_shape=(X.shape[1], X.shape[2])),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)
    return model, X_test, y_test


def predict_and_generate_signals(model, X, y, symbols):
    y_pred = model.predict(X)
    y_label = (y_pred > 0.5).astype(int).flatten()

    # Sinyal tablosu
    result_df = pd.DataFrame({
        'symbol': symbols,
        'true_label': y,
        'predicted': y_label,
        'probability': y_pred.flatten()
    })
    latest_signals = result_df.groupby('symbol').tail(1)
    latest_signals['signal'] = latest_signals['predicted'].map({1: 'AL', 0: 'SAT'})
    latest_signals.to_excel('lstm_sinyalleri.xlsx', index=False)

    # === MODEL PERFORMANSINI EXCELE KAYDET ===
    report_dict = classification_report(y, y_label, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_df.to_excel("lstm_model_performansi.xlsx")

    # Konsola yazdır
    print("\n📁 Sinyaller kaydedildi: lstm_sinyalleri.xlsx")
    print(latest_signals[['symbol', 'signal', 'probability']])
    print("\n📊 Model Performansı:")
    print(report_df.round(3))


def main():
    symbols = get_bist_symbols()
    if not symbols:
        print("Sembol bulunamadı.")
        return
    interval = get_interval_choice()
    bar_ahead = get_bar_ahead(interval)
    df_all = fetch_and_prepare_data(symbols, interval, bar_ahead)
    if df_all.empty:
        print("Veri çekilemedi.")
        return
    X, y, symbol_list, scaler = prepare_lstm_data(df_all, features, bar_ahead)
    model, X_test, y_test = build_and_train_lstm(X, y)
    predict_and_generate_signals(model, X_test, y_test, symbol_list[-len(X_test):])

if __name__ == "__main__":
    main()


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-ajtrfo16
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-ajtrfo16
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17532 sha256=dcaf81dc1b873b8ec70a1185806659e17abf2e0029292612d2e4d7e00c950bf9
  Stored in directory: /tmp/pip-ephem-wheel-cache-a1ao2_9a/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3906aacbf7ac4b0580f489a89de0b24795b649399a9d53890ce7fc1a841335a9
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built tvdatafeed ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5

Veri Alınıyor: 100%|██████████████████████████| 601/601 [05:13<00:00,  1.92it/s]
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 53s 10ms/step - accuracy: 0.5430 - loss: 0.6897 - val_accuracy: 0.5454 - val_loss: 0.6889
Epoch 2/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 52s 10ms/step - accuracy: 0.5443 - loss: 0.6890 - val_accuracy: 0.5484 - val_loss: 0.6873
Epoch 3/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 78s 9ms/step - accuracy: 0.5442 - loss: 0.6878 - val_accuracy: 0.5516 - val_loss: 0.6858
Epoch 4/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 90s 11ms/step - accuracy: 0.5461 - loss: 0.6873 - val_accuracy: 0.5502 - val_loss: 0.6867
Epoch 5/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 75s 10ms/step - accuracy: 0.5471 - loss: 0.6866 - val_accuracy: 0.5512 - val_loss: 0.6859
Epoch 6/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 81s 10ms/step - accuracy: 0.5510 - loss: 0.6863 - val_accuracy: 0.5557 - val_loss: 0.6858
Epoch 7/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.5519 - loss: 0.6856 - val_accuracy: 0.5590 - val_loss: 0.6836
Epoch 8/10
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.5581 - lo

<ipython-input-1-3879200070>:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_signals['signal'] = latest_signals['predicted'].map({1: 'AL', 0: 'SAT'})



📁 Sinyaller kaydedildi: lstm_sinyalleri.xlsx
      symbol signal  probability
63      TERA    SAT     0.407905
443    KAYSE    SAT     0.462512
823     DOCO    SAT     0.426282
1203   VANGD    SAT     0.472348
1583   POLHO    SAT     0.417408
...      ...    ...          ...
43763  METRO     AL     0.539048
44143  SKTAS     AL     0.548311
44523  TDGYO    SAT     0.386577
44903  ALTIN    SAT     0.435063
45278  APX30    SAT     0.453320

[120 rows x 3 columns]

📊 Model Performansı:
              precision  recall  f1-score    support
0                 0.570   0.735     0.642  24614.000
1                 0.519   0.340     0.411  20665.000
accuracy          0.555   0.555     0.555      0.555
macro avg         0.545   0.538     0.527  45279.000
weighted avg      0.547   0.555     0.537  45279.000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful